# Decision Tree Algorithm (ID3)

### Implementing the Decision Tree (ID3) Algorithm Without Using External Libraries

First, let's add the Numpy and Pandas etc. libraries to our workspace.

In [10]:
import numpy as np
import pandas as pd
from pprint import pprint
from collections import Counter
import math

Next, let's add our Iris dataset to our work using the Pandas library, and simplify and shuffle the created dataframe.

In [11]:
df = pd.read_csv("play_tennis.csv")
df = df.sample(frac=1)
df

,Outlook,Temp,Humidity,Wind,PlayTennis
1,Sunny,Hot,High,Strong,No
11,Overcast,Mild,High,Strong,Yes
6,Overcast,Cool,Normal,Strong,Yes
10,Sunny,Mild,Normal,Strong,Yes
3,Rain,Mild,High,Weak,Yes
12,Overcast,Hot,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes
0,Sunny,Hot,High,Weak,No
2,Overcast,Hot,High,Weak,Yes
13,Rain,Mild,High,Strong,No


<img src="https://miro.medium.com/v2/resize:fit:410/format:webp/1*NOn9DS0Q1WsYB6r_cOwg4Q.png" width="200" height="100">

Let's write a function that calculates the entropy formula given above:

In [12]:
def entropy(probs):  
    return sum( [-prob*math.log(prob, 2) for prob in probs])

def entropy_of_list(ls,value):  
    from collections import Counter
    
    total_instances = len(ls)

    cnt = Counter(x for x in ls)

    probs = [x / total_instances for x in cnt.values()]  
    
    return entropy(probs) 

<img src="https://cdn-images-1.medium.com/v2/resize:fit:800/1*Lr8007IV19DRUZZCpMDWAg.png">

Let's write a function that calculates the given Knowledge Acquisition formula above:

In [13]:
def information_gain(df, split_attribute, target_attribute,battr):
    
    df_split = df.groupby(split_attribute) 
    glist=[]
    for gname,group in df_split:
        glist.append(gname) 
    
    glist.reverse()
    nobs = len(df.index) * 1.0   
    df_agg1=df_split.agg({target_attribute:lambda x:entropy_of_list(x, glist.pop())})
    df_agg2=df_split.agg({target_attribute :lambda x:len(x)/nobs})
    
    df_agg1.columns=['Entropy']
    df_agg2.columns=['Proportion']
    
    new_entropy = sum( df_agg1['Entropy'] * df_agg2['Proportion'])
    if battr !='S':
        old_entropy = entropy_of_list(df[target_attribute],'S-'+df.iloc[0][df.columns.get_loc(battr)])
    else:
        old_entropy = entropy_of_list(df[target_attribute],battr)
    return old_entropy - new_entropy

In [14]:
def id3(df, target_attribute, attribute_names, default_class=None,default_attr='S'):
    
    cnt = Counter(x for x in df[target_attribute])
    
    if len(cnt) == 1:
        return next(iter(cnt)) 

    elif df.empty or (not attribute_names):
        return default_class 
    
    else:
        default_class = max(cnt.keys())
        gainz=[]
        for attr in attribute_names:
            ig= information_gain(df, attr, target_attribute,default_attr)
            gainz.append(ig)
        
        index_of_max = gainz.index(max(gainz))
        best_attr = attribute_names[index_of_max] 

        tree = {best_attr:{}}
        remaining_attribute_names =[i for i in attribute_names if i != best_attr]
        
        for attr_val, data_subset in df.groupby(best_attr):
            subtree = id3(data_subset,target_attribute, remaining_attribute_names,default_class,best_attr)
            tree[best_attr][attr_val] = subtree
        return tree

In [16]:
def entropy_dataset(a_list):  
    from collections import Counter

    cnt = Counter(x for x in a_list)   
    num_instances = len(a_list)*1.0 
    
    probs = [x / num_instances for x in cnt.values()]  
    
    return entropy(probs) 

Let's test the tree we created. For this test, let's use the 'play_tennis_test.csv' file.

Looking at the first example, since the 'Outlook' is 'Sunny' and 'Humidity' is 'Normal' according to our tree, the result should be 'PlayTennis = Yes'.

In the second example, since the 'Outlook' is 'Overcast' according to our tree, the result should be 'PlayTennis = Yes'.

In the third example, since the 'Outlook' is 'Rain' and 'Wind' is 'Strong' according to our tree, the result should be 'PlayTennis = No.

In [18]:
total_entropy = entropy_dataset(df['PlayTennis'])
t = df.keys()[-1]
print('Target Attribute is: ', t)
attribute_names = list(df.keys())
attribute_names.remove(t) 
print('Predicting Attributes: ', attribute_names)

from pprint import pprint
tree = id3(df,t,attribute_names)
print("\nThe Final Form of the Generated Decision Tree is as Follows:\n")
pprint(tree)

attribute = next(iter(tree))

Target Attribute is:  PlayTennis
Predicting Attributes:  ['Outlook', 'Temp', 'Humidity', 'Wind']

The Final Form of the Generated Decision Tree is as Follows:

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}


In [20]:
def classify(instance, tree,default=None): 
    attribute = next(iter(tree))  
    if instance[attribute] in tree[attribute].keys(): 
        result = tree[attribute][instance[attribute]]
        if isinstance(result, dict): 
            return classify(instance, result)
        else:
            return result
    else:
        return default
    
df_new=pd.read_csv('play_tennis_test.csv')
df_new['Predicted'] = df_new.apply(classify, axis=1, args=(tree,'?')) 
print(df_new)

    Outlook Temp Humidity    Wind PlayTennis Predicted
0     Sunny  Hot   Normal    Weak          ?       Yes
1  Overcast  Hot     High  Strong          ?       Yes
2      Rain  Hot     High  Strong          ?        No


### Implementing the Decision Tree Algorithm Using the Scikit-Learn Library

In [21]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv("Iris.csv")
df.drop("Id",inplace=True,axis=1)
df = df.sample(frac=1)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
115,6.4,3.2,5.3,2.3,Iris-virginica
97,6.2,2.9,4.3,1.3,Iris-versicolor
64,5.6,2.9,3.6,1.3,Iris-versicolor
23,5.1,3.3,1.7,0.5,Iris-setosa
0,5.1,3.5,1.4,0.2,Iris-setosa
...,...,...,...,...,...
101,5.8,2.7,5.1,1.9,Iris-virginica
14,5.8,4.0,1.2,0.2,Iris-setosa
87,6.3,2.3,4.4,1.3,Iris-versicolor
59,5.2,2.7,3.9,1.4,Iris-versicolor


In [23]:
X = np.array(df.iloc[:,0:-1])
y = np.array(df.iloc[:,-1:]).reshape((-1,))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

In [24]:
dtree = DecisionTreeClassifier(criterion='entropy',max_depth=3)
dtree.fit(X_train,y_train)
y_pred = dtree.predict(X_test)
accuracy_score(y_test, y_pred)

0.9555555555555556